> 在非零和矩阵博弈中，二局中人并不是完全对立的，局中人Ⅰ的所得不再是局中人Ⅱ的所失，这种博弈通常记为 $G=\{S_1,S_2;\boldsymbol A, \boldsymbol B\}$，其中 $\boldsymbol A=(a_{ij})_{m\times n},\ \boldsymbol B=(b_{ij})_{m\times n}$，分别是局中人Ⅰ和Ⅱ的赢得矩阵，故称为二人有限非零和博弈，或***双矩阵博弈***

---
### 16.4.1 非合作的双矩阵博弈的纯策略解
设 $G=\{S_1,S_2;\boldsymbol A, \boldsymbol B\}$ 是一双矩阵博弈，若等式
$$
a_{i^*\!j^*}=\max_i\ \min_j\ a_{ij},\quad b_{i^*\!j^*}=\max_j\ \min_i\ b_{ij}
$$
成立，则记 $v_1=a_{i^*\!j^*}$，并称 $v_1$ 为局中人Ⅰ的赢得值，记 $v_2=b_{i^*\!j^*}$，并称 $v_2$ 为局中人Ⅱ的赢得值，称 $(\alpha_{i^*},\beta_{j^*})$ 为 $G$ 在纯策略下的解（或纳什均衡点），称 $\alpha_{i^*},\beta_{j^*}$ 分别为局中人Ⅰ、Ⅱ的最优纯策略

#### 16.4.2 非合作的双矩阵博弈的混合策略解
如果不存在使上面的等式成立的解，则需要在混合策略意义下求解

**定理**：任何具有有限个纯策略的二人对策（包括零和对策和非零和对策）至少存在一个平衡点（纳什平衡点）

双矩阵博弈混合策略问题的求解归结为求如下线性规划问题的解：
$$
\begin{align*}
&\min\sum\limits_{i=1}^m(x_i+y_i)\\
&{\rm s.t.}
    \begin{cases}
    \sum\limits_{j=1}^ma_{ij}y_j\leq {\boldsymbol x^{\rm T}\boldsymbol A\boldsymbol y},\quad i=1,2,\cdots,m\\[2ex]
    \sum\limits_{i=1}^mb_{ij}x_i\leq {\boldsymbol x^{\rm T}\boldsymbol B\boldsymbol y},\quad j=1,2,\cdots,m\\[2ex]
    \sum\limits_{i=1}^mx_i=1\\
    \sum\limits_{i=1}^my_i=1\\
    x_i, y_i\geq 0,\quad i=1,2,\cdots,m
    \end{cases}
\end{align*}
$$
其中目标函数为虚拟的（显然目标函数的取值恒为 2）

In [1]:
# 此问题非凸
import numpy as np
from scipy.optimize import minimize

A = np.array([[14,13,12],[13,12,12],[12,12,13]])
B = np.array([[13,14,15],[14,15,15],[15,15,14]])

def obj(x):
    return np.sum(x)

def ineq(x):
    xs = x[:3]
    ys = x[3:]
    return np.array([
        -(A @ ys - xs @ A @ ys),
        -(B.T @ xs - xs @ B @ ys),
    ]).flatten()    # ATTENTION!!!

"""
ATTENTION: fun的返回值只能是一维数组
since -(A @ ys - xs @ A @ ys).shape == (3,)
      -(B.T @ xs - xs @ B @ ys).shape == (3,)
so [
    -(A @ ys - xs @ A @ ys),
    -(B.T @ xs - xs @ B @ ys),
].shape == (2, 3), not a 1-d array
you need to flatten() or you get a ValueError
"""

def eq(x):
    xs = x[:3]
    ys = x[3:]
    return np.array([
        np.sum(xs) - 1,
        np.sum(ys) - 1,
    ]).flatten()

x0 = np.ones(6)
cons = [
    {'type': 'ineq', 'fun': ineq},
    {'type': 'eq', 'fun': eq},
]
bd = [(0, 1)]*6
ret = minimize(obj, x0, constraints=cons, bounds=bd)
print(ret)
print('-'*70)
print("最优值为：", ret.fun)
print("最优解为：", ret.x.round(4))

     fun: 2.000000000015837
     jac: array([1., 1., 1., 1., 1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 49
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([5.00000000e-01, 2.76541360e-14, 5.00000000e-01, 3.81793969e-11,
       5.00000000e-01, 5.00000000e-01])
----------------------------------------------------------------------
最优值为： 2.000000000015837
最优解为： [0.5 0.  0.5 0.  0.5 0.5]


In [2]:
x_r = ret.x[:3]
y_r = ret.x[3:]
x_score = x_r @ A @ y_r
y_score = x_r @ B @ y_r
print("甲队得分的期望：", x_score)
print("乙队得分的期望：", y_score)

甲队得分的期望： 12.500000000217035
乙队得分的期望： 14.500000000210559
